In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-recommendation-system/movies.csv
/kaggle/input/movie-recommendation-system/ratings.csv


In [2]:
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import gc
from IPython.display import Markdown
import pickle
from scipy.sparse import csr_matrix
np.random.seed(3213)

# (Análise Complementar) Recomendações baseadas nas similaridades entre os usuários
**Objetivo da Abordagem**
- *Essa técnica utiliza a `Similaridade do Cosseno` para identificar quais usuários tem os gostos mais parecidos. Com isso é possível recomendar filmes que usuários parecidos gostaram, mas que ainda não foram assistidos pelo usuário em questão.*
- *Isso permite gerar recomendações do tipo ***outros usuários também gostaram de…*** com baixo custo computacional, ou seja, apenas guardando os resultados em um aquivo ou um banco de dados, por exemplo.*

**Como Funciona**
- *Filtramos apenas filmes e usuários muitas avaliações atreladas. Assim teremos análises com maior confiabilidade devido ao maior número de amostras individuas, ao mesmo tempo que reduzimos drasticamente o custo computacional.*
- *Após isso, extraímos as avaliações de cada usuário, verificando a similaridade entre elas.*
- *Depois, para cada usuário similar verificamos os filmes que eles mais gostaram.*
- *Quanto maior a nota atribuída, maior a chance de usuários parecidos também gostarem.*
- > Essa abordagem entrega recomendações de alto valor com um custo computacional significativamente menor (ideal para ambientes em nuvem e/ou com recursos limitados).


In [3]:
ratings = pd.read_csv('/kaggle/input/movie-recommendation-system/ratings.csv', engine="pyarrow")
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [4]:
movies = pd.read_csv("/kaggle/input/movie-recommendation-system/movies.csv", engine="pyarrow")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [6]:
ratings['userId'] = ratings['userId'].astype("int32")
ratings['movieId'] = ratings['movieId'].astype("int32")
ratings['rating'] = ratings['rating'].astype("float32")

# Essa coluna sem significância relevante também pode comprometer computacionalmente nossas análises por isso iremos removê-la
ratings.drop('timestamp', axis=1, inplace=True)
print(ratings.dtypes)

userId       int32
movieId      int32
rating     float32
dtype: object


In [7]:
# Filtrar usuários e filmes com pelo menos n avaliação
gc.collect()
df_before = len(ratings)
user_counts = ratings['userId'].value_counts()
movie_counts = ratings['movieId'].value_counts()
filtered_ratings = ratings[
    ratings['userId'].isin(user_counts[user_counts >= 150].index) &
    ratings['movieId'].isin(movie_counts[movie_counts >= 250].index)
]
df_after = len(filtered_ratings)
print("Tamanho do Dataset antes da filtragem: {:d}. Tamanho após a filtragem: {:d}. Redução de {:d} registros".
     format(df_before, df_after, (df_before - df_after)))

Tamanho do Dataset antes da filtragem: 25000095. Tamanho após a filtragem: 17023636. Redução de 7976459 registros


##### Motivos para a redução acima: 
1) A redução acima foi necessária pois os recursos no ambiente do kaggle são limitados e o desempenho poderia ser comprometido com uma matriz tão grande.
2) A matriz gerada seria muito grande e densa, comprometendo também o desempénho em um ambiente na nuvem com recursos ainda mais limitados.
3) Filtrando usuários

In [8]:
filtered_ratings.head(5)

,userId,movieId,rating
70,2,1,3.5
71,2,62,0.5
72,2,110,5.0
73,2,150,4.0
74,2,151,4.5


In [9]:
gc.collect()
user_movie_matrix = filtered_ratings.pivot_table(index='userId',
                                                 aggfunc='mean', columns='movieId', values='rating').fillna(0)
gc.collect()
sparse_matrix = csr_matrix(user_movie_matrix)
gc.collect()
sparse_similarity = cosine_similarity(sparse_matrix, dense_output=False)

In [10]:
# Remove a similaridade entre um usuário e ele mesmo
sparse_similarity.setdiag(0)
sparse_similarity.eliminate_zeros()

In [11]:
rows, cols, sims = [], [], []

for i in range(sparse_similarity.shape[0]):
    gc.collect()
    row = sparse_similarity.getrow(i)
    if row.nnz == 0:
        continue
    top_indices = row.data.argsort()[::-1][:3]  # top-3 maiores
    top_cols = row.indices[top_indices]
    top_sims = row.data[top_indices]
    
    rows.extend([i]*len(top_cols))
    cols.extend(top_cols)
    sims.extend(top_sims)

df_top3 = pd.DataFrame({
    "userId": rows,
    "similarUserId": cols,
    "similarity": sims
})

In [12]:
try:
    del user_movie_matrix, sparse_matrix
except:
    print("Objetos não encontrados no ambiente")

In [13]:
df_top3.loc[df_top3['userId'].isin([12, 13])]

,userId,similarUserId,similarity
36,12,18029,0.728193
37,12,16870,0.712215
38,12,24170,0.660484
39,13,38933,0.651350
40,13,30543,0.637096
41,13,17252,0.617841


## 🔍 Interpretação
**A tabela acima mostra os usuários mais semelhantes com base na `similaridade do cosseno`, uma métrica que compara o perfil de preferências entre usuários. Quanto mais próximo de 1, maior a semelhança.**

1) *Na matriz acima o usuário `12` é mais parecido com o usuário `18029` com uma similaridade de (0.72...), ou seja, quase 73%*
2) *Já para usuário `13` o usuário mais similar é o número `38933` com uma similaridade de (0.65...) mais de 65%*

> ***Essa matriz é útil para sistemas de `recomendação colaborativa`, pois permite sugerir itens com base nas preferências de usuários semelhantes.***

In [14]:
gc.collect()
similar_users = df_top3['similarUserId'].unique()
similar_user_ratings = filtered_ratings[filtered_ratings['userId'].isin(similar_users)]
similar_user_ratings.sort_values(by=["userId", "rating"], ascending=[True, False]).head(3)

,userId,movieId,rating
257,3,50,5.0
261,3,214,5.0
263,3,293,5.0


In [15]:
# Ordena por userId e rating decrescente
sorted_ratings = similar_user_ratings.sort_values(by=["userId", "rating"], ascending=[True, False])

# Agrupa por usuário e pega os 3 primeiros de cada grupo
top3_by_user = sorted_ratings.groupby("userId").head(3).reset_index(drop=True)

# Exibe o resultado
print(top3_by_user)

       userId  movieId  rating
0           3       50     5.0
1           3      214     5.0
2           3      293     5.0
3           8       47     5.0
4           8       69     5.0
...       ...      ...     ...
16288   43917       32     5.0
16289   43917       36     5.0
16290   43918      861     5.0
16291   43918     1907     5.0
16292   43918     3000     5.0

[16293 rows x 3 columns]


In [16]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [17]:
# Filtra o registro dos usuários 12 e 13
filtered = top3_by_user[top3_by_user['userId'].isin([12,13])]

# Pega o Id Dos filmes recomendados
movie_ids = filtered['movieId'].unique()

# Recupera os títulos recomendados
recommended_titles = movies[movies["movieId"].isin(movie_ids)]

recommended_titles

,movieId,title,genres
15,16,Casino (1995),Crime|Drama
49,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
174,176,Living in Oblivion (1995),Comedy
228,231,Dumb & Dumber (Dumb and Dumber) (1994),Adventure|Comedy
292,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
351,356,Forrest Gump (1994),Comedy|Drama|Romance|War


**Interpretação**
+ *Esta tabela mostra os filmes mais bem avaliados pelos usuários.*
+ *Por exemplo, os filme `Casino`, `Usual Suspects` e `Living in Oblivion` receberam a avaliação máxima (5.0) do usuário 12, indicando que essas podem ser boas recomendações para usuários com gostos similares.*
+ *Já o usuário 13 tem entre suas preferências filmes como `Dumb e Dumber`, `Pulp Fiction` e `Forrest Gump`*
+ *Ou seja, esses são os candidatos ideais para aparecer como ***“outros usuários também gostaram de…”*** na interface do sistema.*

In [18]:
top3_by_user.to_parquet("top3_by_user.parquet", compression="snappy")

In [19]:
df_top3.to_parquet("user_similarity_top3.parquet", compression="snappy")

**Nesse ponto, já podemos recomendar filmes baseados em usuários similares.**
**🎬 Funciona assim**
1) **Um Id de usuário é escolhido.**
2) **A tabela de similaridade é consultada para encontrar os usuários mais similares ao usuário atual.**
3) **Os filmes mais bem avaliados pelos usuários similares são recomendados para o usuário atual.**
+ > Essa técnica pode ser utilizada como uma `recomendação complementar` de modelos pré-treinados como SVD (é necessário um hardware mais robusto).
+ > Essa técnica também pode ser utilizada como `recomendação principal` em um sistema de recomendação, sendo ideal para ambientes com recursos limitados.

**Vantagens da Abordagem**
+ 	✅ Muito mais leve que modelos baseados em vetores ou fatoração (como SVD).
+ 	✅ Ideal para ambientes com pouca memória e CPU, como ambientes na nuvem com recursos limitados.
+ 	✅ Fácil de pré-computar e salvar em arquivos simples (CSV, JSON, parquet).
+ 	✅ Permite enriquecer a interface com recomendações sociais (“outros usuários também gostaram de…”).